In [46]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Define custom dataset
class PlantDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_id = str(self.data_frame.iloc[idx, 0])
        img_path = os.path.join(self.root_dir, img_id + '.jpeg')
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        ancillary_data = self.data_frame.iloc[idx, 1:164].values.astype('float32')
        if 'train' in self.root_dir:
            labels = self.data_frame.iloc[idx, 164:].values.astype('float32')
            return [ancillary_data, image], labels
        else:
            return [ancillary_data, image]

# Define transformations
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    # No noise needed since clear images
])

# Load the datasets
train_dataset = PlantDataset(csv_file='data/train.csv', root_dir='data/train_images', transform=data_transforms)
test_dataset = PlantDataset(csv_file='data/test.csv', root_dir='data/test_images', transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [47]:
print(len(train_dataset))
print(len(test_dataset))
print(len(train_loader))
print(len(test_loader))

43363
6391
1356
200


In [48]:
# Define custom dataset for images
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, train=True):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_id = str(self.data_frame.iloc[idx, 0])
        img_path = os.path.join(self.root_dir, img_id + '.jpeg')
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        if self.train:
            labels = self.data_frame.iloc[idx, 164:].values.astype('float32')
            return image, labels
        else:
            return image, img_id

# Define custom dataset for ancillary data
class AncillaryDataset(Dataset):
    def __init__(self, csv_file, train=True):
        self.data_frame = pd.read_csv(csv_file)
        self.train = train

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        ancillary_data = self.data_frame.iloc[idx, 1:164].values.astype('float32')
        if self.train:
            labels = self.data_frame.iloc[idx, 164:].values.astype('float32')
            return ancillary_data, labels
        else:
            img_id = self.data_frame.iloc[idx, 0]
            return ancillary_data, img_id

# Define transformations for images
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load the datasets
train_image_dataset = ImageDataset(csv_file='data/train.csv', root_dir='data/train_images', transform=data_transforms, train=True)
test_image_dataset = ImageDataset(csv_file='data/test.csv', root_dir='data/test_images', transform=data_transforms, train=False)

train_ancillary_dataset = AncillaryDataset(csv_file='data/train.csv', train=True)
test_ancillary_dataset = AncillaryDataset(csv_file='data/test.csv', train=False)

train_image_loader = DataLoader(train_image_dataset, batch_size=32, shuffle=True)
test_image_loader = DataLoader(test_image_dataset, batch_size=32, shuffle=False)

train_ancillary_loader = DataLoader(train_ancillary_dataset, batch_size=32, shuffle=True)
test_ancillary_loader = DataLoader(test_ancillary_dataset, batch_size=32, shuffle=False)

In [49]:
# Define the CNN model for images
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Sequential(
            nn.Linear(self.cnn.fc.in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6)
        )

    def forward(self, x):
        return self.cnn(x)

# Define the fully connected model for ancillary data
class AncillaryModel(nn.Module):
    def __init__(self):
        super(AncillaryModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(163, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6)
        )

    def forward(self, x):
        return self.fc(x)

In [50]:
# Instantiate the models, loss function, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

image_model = ImageModel().to(device)
ancillary_model = AncillaryModel().to(device)

criterion = nn.MSELoss()
image_optimizer = optim.Adam(image_model.parameters(), lr=0.001)
ancillary_optimizer = optim.Adam(ancillary_model.parameters(), lr=0.001)


Using device: cuda


c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [76]:
# Training loop for image model
num_epochs = 10
for epoch in range(num_epochs):
    image_model.train()
    running_loss = 0.0
    for images, labels in train_image_loader:
        images, labels = images.to(device), labels.to(device)
        image_optimizer.zero_grad()
        outputs = image_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        image_optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_image_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Image Model Loss: {train_loss:.4f}')

# # Training loop for ancillary model
# for epoch in range(num_epochs):
#     ancillary_model.train()
#     running_loss = 0.0
#     for ancillary_data, labels in train_ancillary_loader:
#         ancillary_data, labels = ancillary_data.to(device), labels.to(device)
#         ancillary_optimizer.zero_grad()
#         outputs = ancillary_model(ancillary_data)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         ancillary_optimizer.step()
#         running_loss += loss.item()

#     train_loss = running_loss / len(train_ancillary_loader)
#     print(f'Epoch {epoch + 1}/{num_epochs}, Ancillary Model Loss: {train_loss:.4f}')

Epoch 1/10, Image Model Loss: 118905.7461
Epoch 2/10, Image Model Loss: 110258.1904
Epoch 3/10, Image Model Loss: 104731.3162
Epoch 4/10, Image Model Loss: 99524.5344
Epoch 5/10, Image Model Loss: 97730.0542
Epoch 6/10, Image Model Loss: 90986.5182
Epoch 7/10, Image Model Loss: 81622.0190
Epoch 8/10, Image Model Loss: 83884.8307
Epoch 9/10, Image Model Loss: 92388.3497
Epoch 10/10, Image Model Loss: 74668.8965


In [77]:
torch.save(image_model.state_dict(), 'image_model.pth')
torch.save(ancillary_model.state_dict(), 'ancillary_model.pth')

print("Models saved successfully.")

Models saved successfully.


In [78]:
# Prediction and submission for image model
image_model.eval()
image_predictions = []
image_ids = []
with torch.no_grad():
    for images, ids in test_image_loader:
        images = images.to(device)
        outputs = image_model(images)
        image_predictions.extend(outputs.cpu().numpy())
        image_ids.extend([int(id) for id in ids])

# # Prediction and submission for ancillary model
# ancillary_model.eval()
# ancillary_predictions = []
# ancillary_ids = []
# with torch.no_grad():
#     for ancillary_data, ids in test_ancillary_loader:
#         ancillary_data = ancillary_data.to(device)
#         outputs = ancillary_model(ancillary_data)
#         ancillary_predictions.extend(outputs.cpu().numpy())
#         ancillary_ids.extend([int(id) for id in ids])

# Prepare the submission files
image_submission = pd.DataFrame(image_predictions, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
image_submission['id'] = image_ids
image_submission = image_submission[['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]
image_submission.to_csv('20941537_ye_image.csv', index=False)

# ancillary_submission = pd.DataFrame(ancillary_predictions, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
# ancillary_submission['id'] = ancillary_ids
# ancillary_submission = ancillary_submission[['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]
# ancillary_submission.to_csv('20941537_ye_ancillary.csv', index=False)